In [1]:
import datetime
import pandas as pd
import warnings
import sklearn.externals as extjoblib
import joblib
warnings.filterwarnings("ignore")

#from google.cloud import storage

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.linear_model import SGDClassifier
#from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

In [2]:
RANDOM_SEED = 42

In [3]:
df_proc = pd.read_csv('cleaned_non-vectorized_data.csv')
df_proc = df_proc[~df_proc['Text'].isna()]
X = df_proc['Text']

In [4]:
df_proc

,Text,tokens,disgust,joy,anger,surprised,sad,fear,neutral
0,come mert ’ today let u take care lunch enjoy ...,"['come', 'mert', '’', 'today', 'let', 'u', 'ta...",0,0,0,0,0,0,1
1,nxt gt lay 20 staff tech 's latest cutback rb_...,"['nxt', 'gt', 'lay', '20', 'staff', 'tech', ""'...",0,0,0,0,0,0,1
2,layoff 20 workforce 100 employee sf bay area,"['layoff', '20', 'workforce', '100', 'employee...",0,0,0,0,0,0,1
3,today ’ lunch special smoked pork sausage onio...,"['today', '’', 'lunch', 'special', 'smoked', '...",0,0,0,0,0,0,1
4,come mert ’ today grab salmon cake two home co...,"['come', 'mert', '’', 'today', 'grab', 'salmon...",0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
9277,traik01 cdc people warmed u 2 year ago .... sa...,"['traik01', 'cdc', 'people', 'warmed', 'u', '2...",0,0,0,0,1,0,0
9278,sorry ’ promo code share lately 😭 promos autom...,"['sorry', '’', 'promo', 'code', 'share', 'late...",0,0,0,0,1,0,0
9279,poor lad,"['poor', 'lad']",0,0,0,0,1,0,0
9280,one day able bill order tmobile bill sadly tod...,"['one', 'day', 'able', 'bill', 'order', 'tmobi...",0,0,0,0,1,0,0


In [5]:
y = df_proc.loc[:,['disgust', 'joy', 'anger', 'surprised', 'sad', 'fear', 'neutral']]

In [6]:
y

,disgust,joy,anger,surprised,sad,fear,neutral
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
9277,0,0,0,0,1,0,0
9278,0,0,0,0,1,0,0
9279,0,0,0,0,1,0,0
9280,0,0,0,0,1,0,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

### Preprocessing - BOW Approach

In [8]:
vectorizer = TfidfVectorizer(lowercase=True, 
                             stop_words='english', 
                             ngram_range=(1,2),
                             max_df=0.75,
                             min_df=50,
                             norm='l1')

In [9]:
clf = SGDClassifier(random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)

In [10]:
# Transform the features and fit them to the classifier
# multi_out_clf.fit(transformer.fit_transform(vectorizer.fit_transform(X_train)), y_train)

In [11]:
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('multi-classifier', multi_out_clf)
])

In [12]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vectorizer', 'multi-classifier', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__norm', 'vectorizer__preprocessor', 'vectorizer__smooth_idf', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__sublinear_tf', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__use_idf', 'vectorizer__vocabulary', 'multi-classifier__estimator__alpha', 'multi-classifier__estimator__average', 'multi-classifier__estimator__class_weight', 'multi-classifier__estimator__early_stopping', 'multi-classifier__estimator__epsilon', 'multi-classifier__estimator__eta0', 'multi-classifier__estimator__fit_intercept', 'multi-classifier__estimator__l1_ratio', 'multi-classifier__estimator__learning_rate', 'multi-classifier__es

In [15]:
parameters = {
    "vectorizer__max_df": (0.5, 0.75, 1.0),
    "vectorizer__min_df": (50, 100, 200),
    "vectorizer__ngram_range": ((1, 1), (1, 2)), 
    "vectorizer__norm": ('l1', 'l2'),
    "multi-classifier__estimator__loss": ('hinge', 'squared_hinge', 'modified_huber'),
    'multi-classifier__estimator__penalty': ('l1', 'l2', 'elasticnet'), 
    'multi-classifier__estimator__alpha': (0.0001, 0.01, 0.05, 0.1), 
}


In [16]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


GridSearchCV(estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_df=0.75, min_df=50,
                                                        ngram_range=(1, 2),
                                                        norm='l1',
                                                        stop_words='english')),
                                       ('multi-classifier',
                                        MultiOutputClassifier(estimator=SGDClassifier(random_state=42)))]),
             n_jobs=-1,
             param_grid={'multi-classifier__estimator__alpha': (0.0001, 0.01,
                                                                0.05, 0.1),
                         'multi-classifier__estimator__loss': ('hinge',
                                                               'squared_hinge',
                                                               'modified_huber'),
                         'multi-classifier__estimator__penalty'

In [17]:
best_parameters = grid_search.best_estimator_.get_params()

In [18]:
best_parameters

{'memory': None,
 'steps': [('vectorizer',
   TfidfVectorizer(max_df=0.5, min_df=50, norm='l1', stop_words='english')),
  ('multi-classifier',
   MultiOutputClassifier(estimator=SGDClassifier(alpha=0.01, loss='squared_hinge',
                                                 penalty='l1', random_state=42)))],
 'verbose': False,
 'vectorizer': TfidfVectorizer(max_df=0.5, min_df=50, norm='l1', stop_words='english'),
 'multi-classifier': MultiOutputClassifier(estimator=SGDClassifier(alpha=0.01, loss='squared_hinge',
                                               penalty='l1', random_state=42)),
 'vectorizer__analyzer': 'word',
 'vectorizer__binary': False,
 'vectorizer__decode_error': 'strict',
 'vectorizer__dtype': numpy.float64,
 'vectorizer__encoding': 'utf-8',
 'vectorizer__input': 'content',
 'vectorizer__lowercase': True,
 'vectorizer__max_df': 0.5,
 'vectorizer__max_features': None,
 'vectorizer__min_df': 50,
 'vectorizer__ngram_range': (1, 1),
 'vectorizer__norm': 'l1',
 'vectorize

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score

def print_classification_scores(y_test, pred):
    print('Accuracy Score:',accuracy_score(y_test, pred))
    print('Precision Score:',precision_score(y_test, pred, average='micro'))
    print('Recall Score:',recall_score(y_test, pred, average='micro'))
    print('F1 Score:',f1_score(y_test, pred, average='micro'))
    print('AUC Score:',roc_auc_score(y_test, pred, average='micro'))

In [20]:
pred = grid_search.predict(X_test)
print_classification_scores(y_test, pred)

Accuracy Score: 0.4685807150595883
Precision Score: 0.5254604550379198
Recall Score: 0.4534829359513792
F1 Score: 0.4868255959849435
AUC Score: 0.6861219743282816


### Preprocess - Word Embeddings

In [21]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [22]:
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import random
import pandas as pd
import seaborn as sns
import gensim
import nltk
import gensim.downloader
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/m.nguyen.2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
stop_words = set(stopwords.words('english'))

In [25]:
all_tokenized_reviews = []

for review in tqdm(X):
    tokens = [token for token in re.findall(r'\w+', review) if token not in stop_words]
    all_tokenized_reviews.append(tokens)

100%|██████████| 9226/9226 [00:00<00:00, 121557.61it/s]


In [26]:
full_model = Word2Vec(sentences=all_tokenized_reviews, vector_size=100, 
                      window=2, min_count=100, workers=4, seed=RANDOM_SEED)

In [27]:
full_model.save("word2vec_tweets.model")

In [28]:
full_model_kv = full_model.wv

In [29]:
gnews = gensim.downloader.load('word2vec-google-news-300')
glove_small = gensim.downloader.load('glove-wiki-gigaword-100')
glove_big = gensim.downloader.load('glove-wiki-gigaword-300')

In [30]:
def generate_dense_features(tokenized_texts, word_vectors): 
    #HINT: Create an empty list to hold your results 
        #HINT:Iterate through each item in tokenized_text
            #HINT:Create a list that contains current item(s) if found in word_vectors
            #HINT:if the length of this list is greater than zero:
                #HINT:We set this as a feature, this is done by using numpy’s mean function and append it to our results list 
            #HINT:Otherwise: create a vector of numpy zeros using word_vectors.vector_size as the parameter and append it to the results list
    #HINT:Return the results list as a numpy array (data type)

    res = []
    for token in tokenized_texts:
        items_in_vocab = [item for item in token if item in word_vectors]
        if len(items_in_vocab) > 0:
            res.append(np.mean(word_vectors[items_in_vocab], axis=0))
        else:
            res.append(np.zeros(word_vectors.vector_size))
    return np.array(res)


In [31]:
tokenized_train_items = []
for review in tqdm(X_train):
    tokens = [token for token in re.findall(r'\w+', review) if token not in stop_words]
    tokenized_train_items.append(tokens)

100%|██████████| 7380/7380 [00:00<00:00, 23274.65it/s]


In [32]:
tokenized_test_items = []
for review in tqdm(X_test):
    tokens = [token for token in re.findall(r'\w+', review) if token not in stop_words]
    tokenized_test_items.append(tokens)

100%|██████████| 1846/1846 [00:00<00:00, 125371.37it/s]


In [33]:
X_train_wp = generate_dense_features(tokenized_train_items, full_model_kv)

In [34]:
X_test_wp = generate_dense_features(tokenized_test_items, full_model_kv)

In [35]:
clf = SGDClassifier(alpha=0.01, loss='squared_hinge', penalty='l1', 
                    random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.01, loss='squared_hinge',
                                              penalty='l1', random_state=42))

In [36]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Accuracy Score: 0.22643553629469124
Precision Score: 0.4023109243697479
Recall Score: 0.5371669004207573
F1 Score: 0.46006006006006006
AUC Score: 0.6894310807399159


Now we can repeat the same process with the pre-trained embeddings

In [37]:
X_train_wp = generate_dense_features(tokenized_train_items, gnews)

In [38]:
X_test_wp = generate_dense_features(tokenized_test_items, gnews)

In [39]:
clf = SGDClassifier(alpha=0.01, loss='squared_hinge', penalty='l1', 
                    random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.01, loss='squared_hinge',
                                              penalty='l1', random_state=42))

In [40]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Accuracy Score: 0.30985915492957744
Precision Score: 0.5027253668763103
Recall Score: 0.5605423094904161
F1 Score: 0.5300618921308576
AUC Score: 0.7252771827522562


Glove_small

In [41]:
X_train_wp = generate_dense_features(tokenized_train_items, glove_small)

In [42]:
X_test_wp = generate_dense_features(tokenized_test_items, glove_small)

In [43]:
clf = SGDClassifier(alpha=0.01, loss='squared_hinge', penalty='l1', 
                    random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.01, loss='squared_hinge',
                                              penalty='l1', random_state=42))

In [44]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Accuracy Score: 0.22047670639219935
Precision Score: 0.42025420254202545
Recall Score: 0.4791958859280037
F1 Score: 0.4477937964176496
AUC Score: 0.6740317739943273


Glove_big

In [45]:
X_train_wp = generate_dense_features(tokenized_train_items, glove_big)

In [46]:
X_test_wp = generate_dense_features(tokenized_test_items, glove_big)

In [47]:
clf = SGDClassifier(alpha=0.01, loss='squared_hinge', penalty='l1', 
                    random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

MultiOutputClassifier(estimator=SGDClassifier(alpha=0.01, loss='squared_hinge',
                                              penalty='l1', random_state=42))

In [48]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Accuracy Score: 0.2681473456121343
Precision Score: 0.460375816993464
Recall Score: 0.5268817204301075
F1 Score: 0.49138870721604533
AUC Score: 0.7021870347490424
